# 링 1개일 경우 작업하기

In [1]:
import pandas as pd
import os

df = pd.read_excel('sample.xlsx')
df1 = df[['cbl.geo','ring.ring_nm']]
result = pd.DataFrame(columns=['Longitude', 'Latitude'])

# 숫자 추출 함수 정의
def extract_numbers(s):
    return [float(x) for x in s.split()]

In [3]:
for i in range(len(df1)):
    geo = df1['cbl.geo'][i]
    
    geo = geo.replace('LINESTRING (','')
    geo = geo.replace(')','')
    geo = geo.replace(',','')
    
    # 위도와 경도 추출
    numbers = extract_numbers(geo)
    longitude_latitude = [(numbers[i], numbers[i+1]) for i in range(0, len(numbers), 2)]
    # 추출한 위도와 경도 데이터프레임으로 저장
    result_df = pd.DataFrame(longitude_latitude, columns=['Longitude', 'Latitude'])
    result = pd.concat([result, result_df])
    nm1 = df1['ring.ring_nm'][0]
    result['nm'] = nm1

In [4]:
result = result.reset_index(drop=True)

In [5]:
result['Longitude'][0]

127.026486443504

In [6]:
# GPX 텍스트 템플릿
gpx_template = '''<?xml version="1.0" encoding="utf-8"?>
<gpx xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns="http://www.topografix.com/GPX/1/1">
  <trk>
    <trkseg>
      {trkpts}
    </trkseg>
  </trk>
</gpx>
'''

# trkpt 템플릿
trkpt_template = '<trkpt lon="{lon}" lat="{lat}"><ele>-1</ele></trkpt>'

In [8]:
# trkpts 생성
trkpts = ''
for _, row in result.iterrows():
    trkpts += trkpt_template.format(lon=row['Longitude'], lat=row['Latitude'])
# GPX 텍스트 생성
gpx_text = gpx_template.format(trkpts=trkpts)

In [11]:
# nm 값으로 파일명 만들기
file_name = result['nm'].iloc[0]  # 여기서는 첫 번째 값으로 파일명을 만들었습니다.
file_name = file_name.replace('[', '').replace(']', '').replace('/', '_').replace(' ', '_')
gpx_file_name = f"{file_name}.gpx"

# GPX 파일로 저장
with open(gpx_file_name, 'w') as f:
    f.write(gpx_text)

print(f"{gpx_file_name} 파일이 저장되었습니다.")

Mini08{GS}아산_둔포_에버빌(운용337-69).gpx 파일이 저장되었습니다.


# 링 n개 이상 

In [134]:
df = pd.read_excel('sample3.xlsx')
df1 = df[['cbl.geo','ring.ring_nm']]
result = pd.DataFrame(columns=['Longitude', 'Latitude','nm'])

# 숫자 추출 함수 정의
def extract_numbers(s):
    return [float(x) for x in s.split()]

def geo_replace(geo):
    geo = geo.replace('LINESTRING (','')
    geo = geo.replace(')','')
    geo = geo.replace(',','')
    
    return geo

In [136]:
df1

,cbl.geo,ring.ring_nm
0,LINESTRING (127.3837661824834 36.3468641589470...,(DUMMY) PTP_대전_L2_대덕구청VoIP_대전RDC
1,"LINESTRING (127.386631810859 36.3468738313866,...",(DUMMY) PTP_대전_L2_대덕구청VoIP_대전RDC
2,"LINESTRING (127.390128234741 36.3468249716979,...",(DUMMY) PTP_대전_L2_대덕구청VoIP_대전RDC
3,"LINESTRING (127.40628436527 36.3812218745539, ...",(DUMMY) 둔산-송강-01링
4,LINESTRING (127.39191028087181 36.374449966680...,(DUMMY) 둔산-송강-01링
5,LINESTRING (127.39191028087181 36.374449966680...,(DUMMY) 둔산-송강-01링
6,LINESTRING (127.37982703786267 36.374815871429...,(DUMMY) 둔산-송강-01링
7,"LINESTRING (127.340787375175 36.2966995343413,...",(PON이원화)_천년나무아파트
8,LINESTRING (127.34080091445057 36.296720440285...,(PON이원화)_천년나무아파트
9,"LINESTRING (127.337582382389 36.296522548419, ...",(PON이원화)_천년나무아파트


In [137]:
nm_list = df1['ring.ring_nm'].unique()

In [139]:
for i in range(len(nm_list)):
    print(nm_list[i], " == geo 분리 작업중 == ")
    df2 = df1[df1['ring.ring_nm']==nm_list[i]]
    df2 = df2.reset_index(drop=True)
    for j in range(len(df2)):
        geo = geo_replace(df2['cbl.geo'][j])
        # 위도와 경도 추출
        numbers = extract_numbers(geo)
        longitude_latitude = [(numbers[j], numbers[j+1]) for j in range(0, len(numbers), 2)]
        # 추출한 위도와 경도 데이터프레임으로 저장
        result_df = pd.DataFrame(longitude_latitude, columns=['Longitude', 'Latitude'])
        nm1 = df2['ring.ring_nm'][0]
        result_df['nm'] = nm1
        result = pd.concat([result, result_df])
        



(DUMMY) PTP_대전_L2_대덕구청VoIP_대전RDC  == 작업중 == 
(DUMMY) 둔산-송강-01링  == 작업중 == 
(PON이원화)_천년나무아파트  == 작업중 == 


In [140]:
result = result.reset_index(drop=True)

In [141]:
result

,Longitude,Latitude,nm
0,127.383766,36.346864,(DUMMY) PTP_대전_L2_대덕구청VoIP_대전RDC
1,127.383799,36.346874,(DUMMY) PTP_대전_L2_대덕구청VoIP_대전RDC
2,127.383800,36.346874,(DUMMY) PTP_대전_L2_대덕구청VoIP_대전RDC
3,127.383826,36.347121,(DUMMY) PTP_대전_L2_대덕구청VoIP_대전RDC
4,127.383864,36.347120,(DUMMY) PTP_대전_L2_대덕구청VoIP_대전RDC
...,...,...,...
93,127.337582,36.296523,(PON이원화)_천년나무아파트
94,127.337517,36.296443,(PON이원화)_천년나무아파트
95,127.337499,36.295920,(PON이원화)_천년나무아파트
96,127.337499,36.295920,(PON이원화)_천년나무아파트


In [143]:
# GPX 텍스트 템플릿
gpx_template = '''<?xml version="1.0" encoding="utf-8"?>
<gpx xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns="http://www.topografix.com/GPX/1/1">
  <trk>
    <trkseg>
      {trkpts}
    </trkseg>
  </trk>
</gpx>
'''

# trkpt 템플릿
trkpt_template = '<trkpt lon="{lon}" lat="{lat}"><ele>-1</ele></trkpt>'

In [151]:
for k in range(len(nm_list)):
    print(nm_list[k], " == gpx 작성중 == ")
    df3 = result[result['nm'] == nm_list[k]]
    df3 = df3.reset_index(drop=True)
    
    # trkpts 생성
    trkpts = ''
    for _, row in df3.iterrows():
        trkpts += trkpt_template.format(lon=row['Longitude'], lat=row['Latitude'])
    # GPX 텍스트 생성
    gpx_text = gpx_template.format(trkpts=trkpts)

    # nm 값으로 파일명 만들기
    file_name = df3['nm'].iloc[0]  # 여기서는 첫 번째 값으로 파일명을 만들었습니다.
    file_name = file_name.replace('[', '').replace(']', '').replace('/', '_').replace(' ', '_')
    gpx_file_name = f"{file_name}.gpx"

    # GPX 파일로 저장
    with open(gpx_file_name, 'w') as f:
        f.write(gpx_text)

    print(f"{gpx_file_name} 파일이 저장되었습니다.")

(DUMMY) PTP_대전_L2_대덕구청VoIP_대전RDC  == gpx 작성중 == 
(DUMMY)_PTP_대전_L2_대덕구청VoIP_대전RDC.gpx 파일이 저장되었습니다.
(DUMMY) 둔산-송강-01링  == gpx 작성중 == 
(DUMMY)_둔산-송강-01링.gpx 파일이 저장되었습니다.
(PON이원화)_천년나무아파트  == gpx 작성중 == 
(PON이원화)_천년나무아파트.gpx 파일이 저장되었습니다.
